In [17]:
import numpy as np
import pysindy as ps

from scipy.integrate import solve_ivp
from pysindy.utils import linear_damped_SHO

import matplotlib.pyplot as plt

from numpy import genfromtxt

import deepSI
from deepSI import System_data

import csv
import os

In [18]:
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

In [19]:
# training data
dt = 0.01
t_train = np.arange(0, 25, dt)
t_train_span = (t_train[0], t_train[-1])
x0_train = [2, 0]
x_train = solve_ivp(linear_damped_SHO, t_train_span,
                    x0_train, t_eval=t_train, **integrator_keywords).y.T

In [20]:
# differentiate
def gen_diff(x_data, t_data, order=2, diff=None):
    diff = ps.differentiation.FiniteDifference(order=order) if diff is None else diff
    x_dot = diff._differentiate(x_data, t_data)
    return x_dot

In [54]:
def gen_Theta(data, lib="poly", order=3):
    if isinstance(lib, str):
        if lib=="poly":
            from sklearn.preprocessing import PolynomialFeatures

            return PolynomialFeatures(order).fit_transform(data)
        elif lib=="trig":
            from pysindy.feature_library import FourierLibrary

            return np.array(FourierLibrary(order).fit(data).transform(data))
    else:
        from pysindy.feature_library import CustomLibrary

        return np.array(CustomLibrary(library_functions=lib).fit(data).transform(data))

In [55]:

def calculate_regression(theta, x_dot, threshold, max_iterations):
    # Solve theta * xi = x_dot in the least-squares sense.
    xi = np.linalg.lstsq(theta, x_dot, rcond=None)[0]
    n = xi.shape[1]

    for i in range(max_iterations):
        small_indices = np.abs(xi) < threshold
        xi[small_indices] = 0
        for j in range(n):
            big_indices = np.logical_not(small_indices[:, j])
            xi[big_indices, j] = np.linalg.lstsq(theta[:, big_indices],
                                                 x_dot[:, j],
                                                 rcond=None)[0]

    return xi

In [56]:
x_dot_train = gen_diff(x_train, t_train, order=3)
Theta = gen_Theta(x_train, order=1)
xi = calculate_regression(Theta, x_dot_train, 0.01, 1)

In [57]:
xi

array([[  0.54266415,  -1.6964389 ],
       [-83.30041189, 277.65992274],
       [-33.88886531, -21.90275012]])

In [58]:
DATA_PATH = r"C:\Users\20173928\OneDrive - TU Eindhoven\Documents\Master\thesis\mscth\data\\"

WIENER = "WienerHammerBenchmark"
SILVER = "SNLS80mV"
# SILVER = "Schroeder80mV"
# change data set
DATA = SILVER
CSV = ".csv"

PATH = os.path.join(DATA_PATH, DATA+CSV)

# load data
data = genfromtxt(PATH, delimiter=",")
data = data[1:,:-1] # snl

In [59]:
V1 = data[:,0]
V2 = data[:,1]

N = V1.shape[0]
fs = 610.35
dt = 1/fs
t = np.linspace(0,dt*N,N)

silver_data = System_data(u=V1,y=V2)

train, test = silver_data[40000:], silver_data[:40000]

In [60]:
x_train = np.c_[train.u, train.y]
t_train = np.linspace(0,dt*train.y.shape[0],train.y.shape[0])

x_dot_train = gen_diff(x_train, t_train, order=3)
Theta = gen_Theta(x_train, order=3)
xi = calculate_regression(Theta, x_dot_train, 0.1, 1)

In [61]:
xi

array([[ 5.25619432e-01, -1.65198612e+00],
       [-8.22431281e+01,  2.73479044e+02],
       [-3.20677940e+01, -2.12289132e+01],
       [-2.16807497e+01, -9.99347134e+01],
       [ 8.85763345e+00, -1.41088658e+01],
       [ 8.96973000e+00,  1.85929420e+00],
       [ 8.46868965e+02,  2.88229138e+03],
       [-5.56726330e+02,  1.38226434e+03],
       [-6.74886684e+02,  8.44847238e+01],
       [-1.60031315e+02, -1.52824848e+02]])

In [62]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.linear_model import LassoCV, MultiTaskLassoCV

# Generate synthetic data for a damped harmonic oscillator
def generate_data(num_samples, damping_ratio, natural_frequency, dt):
    t = np.arange(0, num_samples * dt, dt)
    x = np.exp(-damping_ratio * natural_frequency * t) * np.cos(np.sqrt(1 - damping_ratio**2) * natural_frequency * t)
    dxdt = -damping_ratio * natural_frequency * x - natural_frequency * np.exp(-damping_ratio * natural_frequency * t) * np.sin(np.sqrt(1 - damping_ratio**2) * natural_frequency * t)
    return t, x, dxdt

# Parameters
num_samples = 1000
damping_ratio = 0.1
natural_frequency = 2 * np.pi
dt = 0.01

# Generate synthetic data
t, x, dxdt = generate_data(num_samples, damping_ratio, natural_frequency, dt)

# Construct library matrix
Theta = np.vstack((x, dxdt)).T
Xi = np.zeros((num_samples, 2))
Xi[:, 0] = dxdt
Xi[:, 1] = -2 * damping_ratio * natural_frequency * dxdt - natural_frequency**2 * x

# Convert to PyTorch tensors
Theta_tensor = torch.tensor(Theta, dtype=torch.float32)
Xi_tensor = torch.tensor(Xi, dtype=torch.float32)

# Define SINDy model
class SINDy(nn.Module):
    def __init__(self, num_features):
        super(SINDy, self).__init__()
        self.linear = nn.Linear(num_features, 2)  # Two terms: x_dot and -2*zeta*omega_n*x_dot - omega_n^2*x

    def forward(self, x):
        return self.linear(x)

# Train the model
model = SINDy(Theta.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 20000
for epoch in range(num_epochs):
    outputs = model(Theta_tensor)
    loss = criterion(outputs, Xi_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {}'.format(epoch+1, num_epochs, loss.item()))

# Extract coefficients
learned_coefficients = model.linear.weight.detach().numpy()

# LASSO regularization for sparsity
lasso = MultiTaskLassoCV()
lasso.fit(Theta, Xi)
lasso_coefficients = lasso.coef_

print("Learned coefficients (PyTorch):", learned_coefficients)
print("LASSO coefficients:", lasso_coefficients)


Epoch [100/20000], Loss: 28.63304901123047
Epoch [200/20000], Loss: 27.174068450927734
Epoch [300/20000], Loss: 25.773475646972656
Epoch [400/20000], Loss: 24.427539825439453
Epoch [500/20000], Loss: 23.134105682373047
Epoch [600/20000], Loss: 21.891368865966797
Epoch [700/20000], Loss: 20.69769287109375
Epoch [800/20000], Loss: 19.551549911499023
Epoch [900/20000], Loss: 18.451499938964844
Epoch [1000/20000], Loss: 17.396162033081055
Epoch [1100/20000], Loss: 16.384220123291016
Epoch [1200/20000], Loss: 15.414421081542969
Epoch [1300/20000], Loss: 14.485552787780762
Epoch [1400/20000], Loss: 13.596447944641113
Epoch [1500/20000], Loss: 12.745987892150879
Epoch [1600/20000], Loss: 11.933093070983887
Epoch [1700/20000], Loss: 11.156712532043457
Epoch [1800/20000], Loss: 10.415834426879883
Epoch [1900/20000], Loss: 9.709476470947266
Epoch [2000/20000], Loss: 9.0366792678833
Epoch [2100/20000], Loss: 8.396519660949707
Epoch [2200/20000], Loss: 7.788087844848633
Epoch [2300/20000], Loss: 7

In [46]:
learned_coefficients

array([[-5.5530779e-07,  9.9999958e-01],
       [-8.8166809e+00, -2.8912848e-01]], dtype=float32)

In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class SINDy(nn.Module):
    def __init__(self, input_dim, output_dim, library_functions):
        super(SINDy, self).__init__()
        self.library_functions = library_functions
        self.output_dim = output_dim
        self.coefficients = nn.Parameter(torch.randn(len(library_functions), output_dim))

    def forward(self, x):
        return torch.matmul(x, self.coefficients)

def generate_library_functions(X, degree):
    # Generate library functions up to a certain degree
    library_functions = []
    for d in range(1, degree + 1):
        for i in range(X.shape[1]):
            library_functions.append(np.power(X[:, i], d))
    library_functions = np.column_stack(library_functions)
    return library_functions

def lasso_loss(parameters, sparsity):
    l1_penalty = torch.norm(parameters, 1)
    return sparsity * l1_penalty

def train_sindy(X, y, degree, sparsity, lr=0.01, epochs=100):
    library_functions = generate_library_functions(X, degree)
    input_dim = library_functions.shape[1]
    output_dim = y.shape[1]

    model = SINDy(input_dim, output_dim, library_functions)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(torch.tensor(library_functions, dtype=torch.float32))
        loss = torch.mean((output - torch.tensor(y, dtype=torch.float32)) ** 2) + lasso_loss(model.coefficients, sparsity)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item()}')

    return model.coefficients.detach().numpy()

# Example usage
# Assuming X is your input data (features) and y is your target data
X = np.array(x_train)
y = np.array(x_train)

degree = 2  # Maximum degree of the polynomial terms
sparsity = 0.1  # Sparsity parameter
coefficients = train_sindy(X, y, degree, sparsity)
print("Coefficients:")
print(coefficients)


import torch
import torch.nn as nn
import torch.optim as optim

class SINDy(nn.Module):
    def __init__(self, input_dim, output_dim, sparsity):
        super(SINDy, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.sparse_loss = nn.L1Loss()
        self.sparsity = sparsity
        
    def forward(self, x):
        return self.linear(x)
    
    def lasso_loss(self, parameters):
        l1_penalty = torch.norm(parameters, 1)
        return self.sparsity * l1_penalty
    
    def fit(self, X, y, lr=0.01, epochs=100):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            output = self(X)
            loss = self.sparse_loss(output, y) + self.lasso_loss(self.linear.weight)
            loss.backward()
            optimizer.step()
            if epoch % 10 == 0:
                print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item()}')

# Example usage
# Assuming X is your input data (features) and y is your target data
X = torch.tensor(x_train, dtype=torch.float32)
y = torch.tensor(x_train, dtype=torch.float32)

input_dim = X.shape[1]  # Number of input features
output_dim = y.shape[1]  # Number of output features
sparsity = 0.1  # Sparsity parameter

model = SINDy(input_dim, output_dim, sparsity)
model.fit(X, y)

coefficients = model.linear.weight.detach().numpy()
print("Coefficients:")
print(coefficients)

import torch
from torch import nn

class LassoRegression(torch.nn.Module):
    def __init__(self, input_size, output_size, alpha):
        super(LassoRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.alpha = alpha
        
    def forward(self, x):
        return self.linear(x)
    
    def l1_penalty(self):
        l1_crit = torch.nn.L1Loss(reduction='sum')
        reg_loss = 0
        for param in self.parameters():
            reg_loss += l1_crit(param, torch.zeros_like(param))
        return self.alpha * reg_loss

# Sample data
X = torch.randn(100, 10)  # 100 samples, 10 features
Y = torch.randn(100, 1)    # 100 samples, 1 output

# Model, criterion, and optimizer
model = LassoRegression(input_size=10, output_size=1, alpha=0.01)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    # Loss
    loss = criterion(outputs, Y) + model.l1_penalty()
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# After training
print('Final Lasso regression coefficients:')
for name, param in model.named_parameters():
    if 'weight' in name:
        print(name, param.data)


IndexError: tuple index out of range